In [2]:
%load_ext nb_black
%config IPCompleter.greedy=True
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

<IPython.core.display.Javascript object>

In [30]:
df_train = pd.read_csv("movie_review_train.csv")

<IPython.core.display.Javascript object>

In [31]:
df_train.head()

,class,text
0,Pos,a common complaint amongst film critics is ...
1,Pos,whew this film oozes energy the kind of b...
2,Pos,steven spielberg s amistad which is bas...
3,Pos,he has spent his entire life in an awful litt...
4,Pos,being that it is a foreign language film with...


<IPython.core.display.Javascript object>

In [32]:
df_train["label"] = df_train["class"].map({"Neg": 0, "Pos": 1})
df_train = df_train.drop("class", axis=1)
df_train.head()

,text,label
0,a common complaint amongst film critics is ...,1
1,whew this film oozes energy the kind of b...,1
2,steven spielberg s amistad which is bas...,1
3,he has spent his entire life in an awful litt...,1
4,being that it is a foreign language film with...,1


<IPython.core.display.Javascript object>

#### What is the size of vocabulary after removing the stop words? Note that the vocabulary size depends only on the training set.

In [25]:
vect = CountVectorizer(stop_words="english")
vect.fit(df_train.text)
len(vect.vocabulary_.keys())

35858

<IPython.core.display.Javascript object>

#### Suppose we don't want to consider those (rare) words which have appeared only in 3% of the documents, or say those (extremely common ones) which have appeared in 80% of the documents.
#### Use CountVectorizer(stop_words='english', min_df=.03, max_df=.8) to create a new vocabulary from the training set. What is the size of the new vocabulary?

In [49]:
X_train, y_train = df_train.text, df_train.label
vect = CountVectorizer(stop_words="english", min_df=0.03, max_df=0.8)
vect.fit(X_train)
X_train_transformed = vect.transform(X_train)
len(vect.vocabulary_.keys())

1643

<IPython.core.display.Javascript object>

#### Suppose we build the vocabulary from the training data using CountVectorizer(stop_words='english', min_df=.03, max_df=.8) and then transform the test data using CountVectorizer(). How many nonzero entries are there in the sparse matrix (corresponding to the test data)? 

##### Note: Test data is provided in a separate CSV file.

In [50]:
df_test = pd.read_csv("movie_review_test.csv")
df_test["label"] = df_test["class"].map({"Neg": 0, "Pos": 1})
df_test = df_test.drop("class", axis=1)
df_test.head()

,text,label
0,films adapted from comic books have had plent...,1
1,every now and then a movie comes along from a...,1
2,you ve got mail works alot better than it des...,1
3,jaws is a rare film that grabs your atte...,1
4,moviemaking is a lot like being the general m...,1


<IPython.core.display.Javascript object>

In [51]:
X_test, y_test = df_test.text, df_test.label
X_test_transformed = vect.transform(X_test)
X_test_transformed

<400x1643 sparse matrix of type '<class 'numpy.int64'>'
	with 51663 stored elements in Compressed Sparse Row format>

<IPython.core.display.Javascript object>

#### Train a Bernoulli Naive Bayes model on the training set and predict the classes of the test set. Each movie review in the test set has been labelled as 'Pos' or 'Neg'. What is the accuracy of the model?

##### Note - Dictionary should be prepared using CountVectorizer(stop_words='english', min_df=.03, max_df=.8)

In [52]:
bnb = BernoulliNB()
bnb.fit(X_train_transformed, y_train)
y_pred_class = bnb.predict(X_test_transformed)

metrics.accuracy_score(y_test, y_pred_class)

0.79

<IPython.core.display.Javascript object>

#### The confusion matrix is a matrix which tabulates

#### True Negative(TN) , False Positive (FP) , False Negative (FN) and True Negative (TN) as follows:

<table border="1"><colgroup><col><col><col></colgroup><thead><tr><th scope="row">&nbsp;</th><th scope="col"><p dir="ltr">Predicted Negative</p><p dir="ltr">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⇓</p></th><th scope="col"><p dir="ltr">Predicted Positive</p><p dir="ltr">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⇓</p></th></tr></thead><tbody><tr><th scope="row"><p dir="ltr">Actual Negative ⇒</p></th><td><p dir="ltr">TN</p></td><td><p dir="ltr">FP</p></td></tr><tr><th scope="row"><p dir="ltr">Actual Positive &nbsp;&nbsp;⇒</p></th><td><p dir="ltr">FN</p></td><td><p dir="ltr">TP</p></td></tr></tbody></table>

#### Run metrics.confusion_matrix(actual class of test data, predicted class of test data). How many reviews are actually negative but have been classified as positive by the model?

##### Note :

1. Dictionary should be  prepared using CountVectorizer(stop_words='english',min_df=.03,max_df=.8)

2. Remember that we have tagged negative as 0 and positive as 1 and If needed, look up the documentation of confusion_matrix to understand which values in the cells correspond to positives/negatives.

3. The CF docs mention that C{i, j} is the number which is known to be in class i but are predicted in class j. In this case, {0,1} is thus actually 0 (negative) and predicted 1 (positive).

In [55]:
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
TP = confusion[1, 1]
FP

23

<IPython.core.display.Javascript object>